# Demo of Negational Symmetry of QNNs

### Load packages.

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_quantum as tfq

import models
import utils

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

NUM_QUBITS = 16

tf.random.set_seed(2021)

### Load MNIST data.

In [2]:
x_train, y_train, x_test, y_test = utils.get_mnist()

## Part 1 Validation of Negational Symmetry
### Part 1.1 Binary Pattern Classication

In [ ]:
# Load quantum data with original test set.
x_train_q, y_train_q = utils.prepare_quantum_data(
    x_train, y_train, 
    num_qubits=16,
    a=3,
    b=6,
    invert=False)

x_test_q, y_test_q = utils.prepare_quantum_data(
    x_test, y_test,
    num_qubits=16,
    a=3,
    b=6,
    invert=False)

In [ ]:
# Train a QNN (XX-ZZ) and evaluate it. The architecture of QNN is defined
# as a list e.g.  ['XX', 'ZZ'],  ['XX', 'ZZ', 'XX'].
arch = ['XX', 'ZZ']
model = models.create_qnn(num_qubits=NUM_QUBITS, arch=arch)

_ = model.fit(
    x_train_q, y_train_q,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_data=(x_test_q, y_test_q))

#### Evaluate on the original test data

In [ ]:
_ = model.evaluate(x_test_q, y_test_q)

#### Evaluate on the negational test data.

In [ ]:
# Load quantum data with negational test set.
x_train_q_inv, y_train_q_inv = utils.prepare_quantum_data(
    x_train, y_train,
    num_qubits=16,
    a=3,
    b=6,
    invert=True)


x_test_q_inv, y_test_q_inv = utils.prepare_quantum_data(
    x_test, y_test,
    num_qubits=16,
    a=3,
    b=6,
    invert=True)

In [ ]:
_ = model.evaluate(x_test_q_inv, y_test_q_inv)

### Part 1.2 Evaluate negational symmetry numerically.

#### Theorem 1 $f_{\theta} (\textbf{x}) = f_{\theta} (\tilde{\textbf{x}})$

In [ ]:
# Here, we provide a pre-trained weights for QNN (XX-ZZ) for convenience. 
arch = ['XX', 'ZZ']
model = models.create_qnn(num_qubits=NUM_QUBITS, arch=arch)
weights = np.load('weights.npy')
model.set_weights([weights])

logits_ori = model(x_test_q, training=False)
logits_neg = model(x_test_q_inv, training=False)

logits_ori_np = logits_ori.numpy()
logits_neg_np = logits_neg.numpy()

diff = logits_ori_np - logits_neg_np

print('mean: {}'.format(np.mean(diff)))
print('std: {}'.format(np.std(diff)))

#### Theorem 2 $g_{\theta} (\textbf{x}) = g_{\theta} (\tilde{\textbf{x}})$

In [ ]:
from numpy import linalg as LA
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine

model = models.create_qnn_feat(num_qubits=16, arch=['XX', 'ZZ'])
weights = np.load('weights.npy')
model.set_weights([weights])

feats = model(x_test_q, training=False)
feats_np = feats.numpy()
feats_inv = model(x_test_q_inv, training=False)
feats_inv_np = feats_inv.numpy()

total_norm = 0
r = []
cos = []
for i in range(len(x_test_q)):
    total_norm += LA.norm(feats_np[i] + feats_inv_np[i])
    r.append(pearsonr(feats_np[i], feats_inv_np[i]))
    cos.append(1 - cosine(feats_np[i], feats_inv_np[i]))
    
print('total norm: {}'.format(total_norm))
print('avg pearson r coefficients: {}'.format(np.mean(r)))
print('avg cosine similarities: {}'.format(np.mean(cos)))

### Part 1.3 t-SNE Visualization.

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

tsne = TSNE(n_components=2, random_state=0, perplexity=100)

In [ ]:
# t-SNE visualization of the original test set
tsne_obj = tsne.fit_transform(feats_np)
tsne_df = pd.DataFrame({'X': tsne_obj[:, 0],
                        'Y': tsne_obj[:, 1],
                        'Class': ((y_test_q + 1) // 2).astype(np.int)})

plt.figure(figsize=(8, 6))
sns.scatterplot(x="X", y="Y", hue="Class", legend="full", data=tsne_df)
plt.legend(loc="best")
plt.legend(fontsize=12)

In [ ]:
# t-SNE visualization of the negational test set
tsne_inv_obj = tsne.fit_transform(feats_inv_np)
tsne_inv_df = pd.DataFrame({'X': tsne_inv_obj[:, 0],
                        'Y': tsne_inv_obj[:, 1],
                        'Class': ((y_test_q + 1) // 2).astype(np.int)})

plt.figure(figsize=(8, 6))
sns.scatterplot(x="X", y="Y", hue="Class", legend="full", data=tsne_inv_df)
plt.legend(loc="best")
plt.legend(fontsize=12)

### Part 1.4 Drawback of Negational Symmetry.

In [ ]:
x_train_q_2 = tf.concat([x_train_q, x_train_q_inv], 0)
y_train_q_2 = np.concatenate([np.ones_like(y_train_q), -1 * np.ones_like(y_train_q_inv)], 0)

x_test_q_2 = tf.concat([x_test_q, x_test_q_inv], 0)
y_test_q_2 = np.concatenate([np.ones_like(y_test_q), -1 * np.ones_like(y_test_q_inv)], 0)

In [ ]:
arch = ['XX', 'ZZ']
model = models.create_qnn(num_qubits=NUM_QUBITS, arch=arch)

_ = model.fit(
    x_train_q_2, y_train_q_2,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_data=(x_test_q_2, y_test_q_2))

## Part 2 Comparison with Classical Models.
### Part 2.1 Compare with DNNs.

In [3]:
# Load classical data with original test set.
x_train_c, y_train_c = utils.prepare_classical_data(
    x_train, y_train,
    num_qubits=16,
    a=3,
    b=6,
    invert=False)

x_test_c, y_test_c = utils.prepare_classical_data(
    x_test, y_test,
    num_qubits=16,
    a=3,
    b=6,
    invert=False)
    

In [ ]:
# Train a DNN and evaluate it.
model = models.create_dnn(num_qubits=NUM_QUBITS)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

_ = model.fit(
    x_train_c, y_train_c,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_data=(x_test_c, y_test_c))

#### Evaluate on the original test data.

In [ ]:
_ = model.evaluate(x_test_c, y_test_c)

#### Evaluate on the negational test data.

In [4]:
# Load classical data with negational test set.
x_train_c_inv, y_train_c_inv = utils.prepare_classical_data(
    x_train, y_train,
    num_qubits=16,
    a=3,
    b=6,
    invert=True)

x_test_c_inv, y_test_c_inv = utils.prepare_classical_data(
    x_test, y_test,
    num_qubits=16,
    a=3,
    b=6,
    invert=True)

In [ ]:
_ = model.evaluate(x_test_c_inv, y_test_c_inv)

# Part 2.2 Compare with CNNs.

In [ ]:
model = models.create_cnn(num_qubits=NUM_QUBITS)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

_ = model.fit(
    x_train_c, y_train_c,
    batch_size=32,
    epochs=10,
    verbose=1,
    validation_data=(x_test_c, y_test_c))

#### Evaluate on the original test data.

In [ ]:
_ = model.evaluate(x_test_c, y_test_c)

#### Evaluate on the negational test data.

In [ ]:
_ = model.evaluate(x_test_c_inv, y_test_c_inv)

### Compare with SVMs.

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

num_train = len(x_train_c)
x_train_c_np = tf.reshape(x_train_c, [num_train, -1]).numpy()
num_test = len(x_test_c)
x_test_c_np = tf.reshape(x_test_c, [num_test, -1]).numpy()

x_test_c_inv_np = tf.reshape(x_test_c_inv, [num_test, -1]).numpy()

clf = svm.SVC(C=C, kernel='rbf')
clf.fit(x_train_bin, y_train)

y_pred = clf.predict(x_test_bin)
print('Accuracy on the original test set: {}'.format(accuracy_score(y_test_c, y_pred)))
  
y_pred = clf.predict(x_test_bin_in)
print('Accuracy on the negational test set: {}', accuracy_score(y_test_c_inv, y_pred)))